# Spark SQL review

In [10]:
import random

N_ELEM = 100
# NOTE: randint includes the specified boudaries

def gener_val():
    return random.sample(range(0, 100), 5)

ids = ['first','second', 'third', 'fourth', 'fifth']
labels = ['aaa', 'bbb', 'c', 'ccc'] 
def gener_id():
    return ids[random.randint(0, 4)]

collection = []
for i in range(N_ELEM):
    new_el = gener_val()
    new_el.insert(0, gener_id())
    new_el.append(labels[random.randint(0, 3)])
    
    collection.append(new_el)


In [11]:
cols = ['ID', 'a', 'b', 'c', 'd', 'e', 'label']

In [12]:
in_DF = spark.createDataFrame(collection, cols)
in_DF.createOrReplaceTempView('starting')
in_DF.show(10)

+------+---+---+---+---+---+-----+
|    ID|  a|  b|  c|  d|  e|label|
+------+---+---+---+---+---+-----+
| first|  0| 68| 42| 95| 65|  bbb|
| first| 34| 83| 39| 88|  9|  ccc|
| fifth| 59| 16| 15| 96| 64|    c|
| fifth| 92| 43| 86| 34| 17|  bbb|
| fifth| 26| 96| 57| 68| 35|    c|
| first| 24| 81| 97| 88| 51|  ccc|
| fifth| 75| 28| 91| 29| 61|  ccc|
| third| 32|  9| 15| 48| 78|    c|
|fourth| 93| 29| 17| 28| 80|  aaa|
| fifth| 38| 87| 80| 63| 60|    c|
+------+---+---+---+---+---+-----+
only showing top 10 rows



In [13]:
# Attributes:
print(in_DF.columns)
print(in_DF.ID)

['ID', 'a', 'b', 'c', 'd', 'e', 'label']
Column<b'ID'>


## Aggregate functions without groupby

In [31]:
# Can use aggregate functions WITHOUT GROUPBY
in_DF.agg({'a':'sum','a':'min'}).show()

+------+
|min(a)|
+------+
|     1|
+------+



In [28]:
# It is the same as using selectExpr() passing the column to the 
# aggregate function:
in_DF.selectExpr('sum(a)').show()

+------+
|sum(a)|
+------+
|  5482|
+------+



In [14]:
in_DF.selectExpr("ID", "CAST(a AS float) AS flt_a").show(10)

+------+-----+
|    ID|flt_a|
+------+-----+
| first| 98.0|
|fourth|  5.0|
| third| 23.0|
| third| 45.0|
| fifth| 81.0|
|second| 82.0|
|second| 79.0|
|second| 50.0|
|fourth| 88.0|
| first| 47.0|
+------+-----+
only showing top 10 rows



In [15]:
in_DF.selectExpr("avg(a)").show(10)

+------+
|avg(a)|
+------+
| 53.22|
+------+



In [16]:
# String indexer and index to string
from pyspark.ml.feature import StringIndexer, IndexToString

str2ind = StringIndexer(inputCol='label', outputCol='ind_label').fit(in_DF)
no_cat_DF = str2ind.transform(in_DF)
no_cat_DF.show(10)

ind2str = IndexToString(inputCol='ind_label', outputCol='label_2')
ret_DF = ind2str.transform(no_cat_DF)    # no need to fit!!!
ret_DF.show(10)

+------+---+---+---+---+---+-----+---------+
|    ID|  a|  b|  c|  d|  e|label|ind_label|
+------+---+---+---+---+---+-----+---------+
| first| 98|  6| 63|  2| 15|    c|      3.0|
|fourth|  5| 14| 64| 36| 93|    c|      3.0|
| third| 23| 67| 90| 24| 64|  aaa|      1.0|
| third| 45| 35| 92|  0| 90|  ccc|      2.0|
| fifth| 81| 90| 57| 69| 78|  bbb|      0.0|
|second| 82| 87| 57|  8| 12|  ccc|      2.0|
|second| 79| 15| 82| 12| 56|  bbb|      0.0|
|second| 50| 34| 58| 65| 59|  bbb|      0.0|
|fourth| 88| 21| 90| 60| 62|  aaa|      1.0|
| first| 47| 73| 88| 50| 81|  ccc|      2.0|
+------+---+---+---+---+---+-----+---------+
only showing top 10 rows

+------+---+---+---+---+---+-----+---------+-------+
|    ID|  a|  b|  c|  d|  e|label|ind_label|label_2|
+------+---+---+---+---+---+-----+---------+-------+
| first| 98|  6| 63|  2| 15|    c|      3.0|      c|
|fourth|  5| 14| 64| 36| 93|    c|      3.0|      c|
| third| 23| 67| 90| 24| 64|  aaa|      1.0|    aaa|
| third| 45| 35| 92|  0| 9

## 1

In [17]:
# Spark SQL
sec_DF = in_DF.groupBy('ID').agg({'*': 'count'}).sort('count(1)', ascending=False)
sec_DF.show()

[Stage 27:>                                                     (0 + 134) / 200]

+------+--------+
|    ID|count(1)|
+------+--------+
| first|      24|
|fourth|      23|
| fifth|      20|
| third|      18|
|second|      15|
+------+--------+



In [18]:
# SQL
sec_DF_2 = spark.sql("""
                        SELECT ID, count(*) AS counts
                        FROM starting
                        GROUP BY ID
                        ORDER BY -1*counts
                    """)

sec_DF_2.show()

[Stage 28:======>                                                 (8 + 64) / 72]

+------+------+
|    ID|counts|
+------+------+
| first|    24|
|fourth|    23|
| fifth|    20|
| third|    18|
|second|    15|
+------+------+



## 2 - UDFs

In [19]:
# UDF
spark.udf.register('myTest', lambda c1, c2: (c1+c2)/2)

<function __main__.<lambda>(c1, c2)>

In [20]:
in_DF.selectExpr('ID', 'myTest(a, b) AS avg_ab').show(10)

+------+------+
|    ID|avg_ab|
+------+------+
| first|  52.0|
|fourth|   9.5|
| third|  45.0|
| third|  40.0|
| fifth|  85.5|
|second|  84.5|
|second|  47.0|
|second|  42.0|
|fourth|  54.5|
| first|  60.0|
+------+------+
only showing top 10 rows



## 3 - Extracting a single column

Goal: get a single column of the DF as local Python list.

In [21]:
listOfRows = in_DF.select('a').collect()

print([el['a'] for el in listOfRows])

[98, 5, 23, 45, 81, 82, 79, 50, 88, 47, 62, 52, 93, 90, 44, 66, 63, 85, 40, 87, 2, 58, 91, 99, 9, 94, 35, 51, 69, 76, 83, 37, 51, 52, 71, 24, 44, 62, 40, 51, 26, 43, 99, 76, 30, 79, 92, 35, 67, 70, 71, 42, 12, 75, 53, 52, 52, 96, 75, 28, 18, 39, 67, 44, 34, 26, 41, 17, 25, 12, 49, 9, 9, 89, 97, 15, 1, 36, 32, 99, 13, 28, 93, 62, 78, 18, 3, 64, 69, 83, 78, 28, 87, 55, 41, 55, 29, 26, 16, 85]


## 4 - Counting the elements which satisfy a certain condition

Possibilities: 

* `count(col_name > number)` $\rightarrow$ It simply counts the number of rows...
* `sum(CAST(col_name > number AS int))` $\rightarrow$ **CORRECT**

In [20]:
# Get elements having 'a' > 'b' and numbers of 'aaa' > 3


# First method: IT DOES NOT WORK
spark.sql("""
        SELECT ID, count(a), sum(CAST(label == 'aaa' AS int))
        FROM starting
        WHERE a > b
        GROUP BY ID
        HAVING count(label == 'aaa') > 9
""").show(20)

+------+--------+-------------------------------+
|    ID|count(a)|sum(CAST((label = aaa) AS INT))|
+------+--------+-------------------------------+
|fourth|       8|                              2|
| fifth|       8|                              2|
|second|      14|                              7|
| third|      13|                              1|
| first|      11|                              4|
+------+--------+-------------------------------+



In [18]:
# Second
spark.sql("""
        SELECT ID, sum(a), sum(b), sum(CAST(label == 'aaa' AS int))
        FROM starting
        WHERE a > b
        GROUP BY ID
        HAVING sum(CAST(label == 'aaa' AS int)) > 2
""").show(20)

+------+------+------+-------------------------------+
|    ID|sum(a)|sum(b)|sum(CAST((label = aaa) AS INT))|
+------+------+------+-------------------------------+
|second|  1036|   586|                              7|
| first|   834|   370|                              4|
+------+------+------+-------------------------------+



In [ ]:
in_DF.filter("a > b").show(100)

## Using distinct in agg

In [26]:
spark.sql("""
        SELECT ID, count(distinct(a)) 
        FROM starting
        WHERE a > b
        GROUP BY ID
""").show()

+------+-----------------+
|    ID|count(DISTINCT a)|
+------+-----------------+
|fourth|                8|
| fifth|                8|
|second|               12|
| third|               11|
| first|               10|
+------+-----------------+



## On SQL queries - simulations doubts

In [5]:
# 'starting' table

spark.udf.register('myFunc', lambda n: int(n%2))

# TO INCLUDE NEW COLUMNS IN GROUP BY MUST USE ALIAS!
spark.sql("""
        SELECT ID, myFunc(a) AS new_a, sum(b)
        FROM starting
        WHERE label == 'aaa' OR label == 'bbb'
        GROUP BY ID, new_a
""").show(20)



+------+-----+------+
|    ID|new_a|sum(b)|
+------+-----+------+
| third|    0|   353|
| first|    1|   186|
| fifth|    1|   247|
|fourth|    0|   211|
|second|    0|   212|
|fourth|    1|   447|
| first|    0|   252|
|second|    1|   519|
| fifth|    0|   163|
| third|    1|   401|
+------+-----+------+



In [6]:
# HAVING columns not in select
spark.sql("""
        SELECT ID, myFunc(a) AS new_a
        FROM starting
        WHERE label == 'aaa' OR label == 'bbb'
        GROUP BY ID, new_a
        HAVING sum(b) > 200
""").show(20)


+------+-----+
|    ID|new_a|
+------+-----+
| third|    0|
| fifth|    1|
|fourth|    0|
|second|    0|
|fourth|    1|
| first|    0|
|second|    1|
| third|    1|
+------+-----+



In [7]:
# NO NEED TO INCLUDE GROUP BY COLS IN SELECT
spark.sql("""
        SELECT sum(b)
        FROM starting
        WHERE label == 'aaa' OR label == 'bbb'
        GROUP BY ID, myFunc(a)
""").show(20)

+------+
|sum(b)|
+------+
|   353|
|   186|
|   247|
|   211|
|   212|
|   447|
|   252|
|   519|
|   163|
|   401|
+------+



In [19]:
# Column name in GROUP BY must be the alias - HAVING columns don't have to appear in SELECT
# count(distinct(col)) is allowed
spark.sql("""
        SELECT myFunc(a) AS func_a, sum(b) AS sum_b
        FROM starting
        WHERE label == 'aaa' OR label == 'bbb'
        GROUP BY ID, func_a
        HAVING count(distinct(label)) == 2
        ORDER BY -1*sum_b
""").show(20)

+------+-----+
|func_a|sum_b|
+------+-----+
|     1|  514|
|     0|  396|
|     1|  335|
|     0|  321|
|     0|  277|
|     1|  260|
|     0|  181|
|     1|   66|
+------+-----+



In [24]:
my_DF1 = spark.sql("""
        SELECT ID
        FROM starting
        WHERE label == 'aaa' OR label == 'bbb'
        GROUP BY ID
        HAVING sum(b) > 200
""")

my_DF1.createOrReplaceTempView('t1')

my_DF1.show(20)

+------+
|    ID|
+------+
|fourth|
| fifth|
| third|
| first|
+------+



In [33]:
spark.udf.register('checkFunc', lambda lab: lab.startswith('f'))

spark.sql("""
        SELECT starting.ID, max(starting.a)
        FROM starting, t1
        WHERE starting.ID == t1.ID AND starting.label == 'aaa' AND checkFunc(t1.ID) == True
        GROUP BY starting.ID
""").show()

23/02/03 07:54:18 WARN analysis.SimpleFunctionRegistry: The function checkfunc replaced a previously registered function.


+------+------+
|    ID|max(a)|
+------+------+
|fourth|    93|
| fifth|    86|
| first|    81|
+------+------+

